In [1]:
import os
import importlib
from PIL import Image
import torch

import clip

os.chdir('/home/chenyun/work1/Clip')
!pwd
!echo $CUDA_VISIBLE_DEVICES
!nvidia-smi

/home/chenyun/work1/Clip
0
Sat Jul 24 01:49:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  Off  | 00000000:02:00.0 Off |                  N/A |
| 20%   47C    P0    70W / 250W |      3MiB / 12212MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX TIT...  Off  | 00000000:03:00.0 Off |     

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print('model ready.')

model ready.


In [5]:
from dtd2.applications.fine_grained_classification.cub_dataset import CUBDataset
cub_dataset = CUBDataset(data_path='data/CUB_200_2011')

CUB dataset ready.


In [16]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cub_dataset.class_names]).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

In [35]:
import random
result = dict()

for img_idx in cub_dataset.img_splits['test']:
    img_data = cub_dataset.img_data_list[img_idx]
    img = cub_dataset.load_img(img_idx)
    gt_label = img_data['class_label']
    
    image_input = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        values, indices = similarity[0].topk(5)
        gt_rank = torch.sum(similarity[0] > similarity[0, gt_label])
    result[img_idx] = {'top_5_labels': indices.detach().cpu().numpy(), 
                      'top_5_probs': values.detach().cpu().numpy(), 
                      'gt_rank':gt_rank.detach().cpu().numpy()}

In [36]:
import numpy as np

def evaluate(predicts):
    count = len(predicts)
    metrics = {'mrr': 0, 'acc_top1': 0, 'acc_top5': 0, 'acc_top10': 0}
    for p in predicts.values():
        r = p['gt_rank']
        metrics['mrr'] += 1.0 /(1 + r)
        metrics['acc_top1'] += int(r == 0)
        metrics['acc_top5'] += int(r < 5)
        metrics['acc_top10'] += int(r < 10)
    for k, v in metrics.items():
        metrics[k] = v / count
        print(k, metrics[k])
   
    

In [38]:
evaluate(result)

mrr 0.5193503505212196
acc_top1 0.38919571971004485
acc_top5 0.6853641698308595
acc_top10 0.7811529168104936
